## .필요한 라이브러리 import

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pymongo

## .pymongo와 연결, 데이터 넣을 DB, Collection  

In [33]:
conn = pymongo.MongoClient()
cine_db = conn.cine
movie_info = cine_db.movie

## .웹크롤링 코드 작성

In [46]:
url = "http://www.cine21.com/rank/boxoffice/domestic_content"
movies_info = []
header = {}
header['genre'] = 'all'
header['period'] = '2019-04-09'
for n in range(1,6):
    header['page'] = n
    res = requests.post(url, data=header)
    soup = bs(res.content, 'html.parser')

    movies = soup.select("ul.boxoffice_list li.boxoffice_li")
    for index, movie in enumerate(movies):
        name = movie.select_one('a > div.mov_name').text
        audience = movie.select_one('div.people_num').text
        audience = int(audience.split("|")[1].replace(",",''))
        # ranking도 문자열이므로 int형으로 바꿔주기
        ranking = int(movie.select_one('a span.grade.num1').text)
        link = movie.select_one('a')['href']
        link = f"http://www.cine21.com{link}"
        # 재귀적 접속 링크 요청 시작
        res = requests.get(link)
        soup = bs(res.content, 'html.parser')
        movie_details = {}
        year = soup.select_one("div.mov_info p.sub_info span:nth-child(1)").text
        country = soup.select_one("div.mov_info p.sub_info span:nth-child(2)").text
        proper_age = soup.select_one("div.mov_info p.sub_info span:nth-child(3)")
        if proper_age is None:
            proper_age = "연령대없음"
        else:
            proper_age = proper_age.text
        genre = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(1)").text
        runtime = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(2)")
        if runtime is None:
            runtime = "상영시간없음"
        else:
            runtime = runtime.text
            runtime = runtime.split(':')[1].strip()
        released_date = soup.select_one("div.mov_info p:nth-child(5) span")
        if released_date is None:
            released_date = "정보없음"
        else:
            released_date = released_date.text
            released_date = released_date.split(':')[1].strip()
        directors = soup.select("div.mov_info p:nth-child(6) a")
        directors_list = []
        for director in directors:
            director = director.text
            directors_list.append(director)
        # 인덱스 -1은 'more'을 나타내므로 마지막 인덱스를 제외한 나머지만 리스트로 담기
        actors = soup.select("div.mov_info p:nth-child(7) a")[:-1]
        actors_list = []
        for actor in actors:
            actor = actor.text
            actors_list.append(actor)

        cine_reviews = soup.select_one("ul.mov_rating li:nth-child(1) span")
        if cine_reviews is None:
            cine_reviews = "평점없음"
        else:
            cine_reviews = float(cine_reviews.text)
        people_reviews = soup.select_one("ul.mov_rating li:nth-child(2) span")
        if people_reviews is None:
            people_reviews = "평점없음"
        else:
            people_reviews = float(people_reviews.text)

        movie_details['영화이름'] = name
        movie_details['관객수'] = audience
        movie_details['순위'] = ranking
        movie_details['개봉연도'] = year
        movie_details['국가'] = country
        movie_details['관람적절연령'] = proper_age
        movie_details['장르'] = genre
        movie_details['상영시간'] = runtime
        movie_details['개봉날짜'] = released_date
        movie_details['감독'] = directors_list
        movie_details['출연배우'] = actors_list
        movie_details['씨네21평점'] = cine_reviews
        movie_details['네티즌평점'] = people_reviews

        movies_info.append(movie_details)




## . pymongo DB에 데이터 잘 들어가는지 확인

- Robo 3T 앱을 이용해서 잘 들어갔는지 확인

In [35]:
movie_info.insert_one({'result':'mongo'})

In [39]:
movie_info.delete_many({})

## .pymongo DB에 크롤링한 데이터 입력

In [47]:
movie_info.insert_many(movies_info)